In [3]:
import scipy.io
import os, importlib
# import matplotlib.pyplot as plt
# import statistics
# import scipy.stats
# import matplotlib.patches as mpatches
# import seaborn as sns; sns.set()
# import tkinter as tk
# from matplotlib import gridspec
# import matplotlib.colors
import numpy as np
import pandas as pd
import os.path, time
import pickle

In [ ]:

def loadmat(filename):
    '''
    this function should be called instead of direct spio.loadmat
    as it cures the problem of not properly recovering python dictionaries
    from mat files. It calls the function check keys to cure all entries
    which are still mat-objects
    '''
    data = scipy.io.loadmat(filename, struct_as_record=False, squeeze_me=True)
    return check_keys(data)

def check_keys(temp_dict):
    '''
    checks if entries in dictionary are mat-objects. If yes
    todict is called to change them to nested dictionaries
    '''
    for key in temp_dict:
        if isinstance(temp_dict[key], scipy.io.matlab.mio5_params.mat_struct):
            temp_dict[key] = todict(temp_dict[key])
          
    return temp_dict        

def todict(matobj):
    '''
    A recursive function which constructs from matobjects nested dictionaries
    '''
    temp_dict = {}
    for strg in matobj._fieldnames:
        elem = matobj.__dict__[strg]
        if isinstance(elem, scipy.io.matlab.mio5_params.mat_struct):
            temp_dict[strg] = todict(elem)
        else:
            temp_dict[strg] = elem
    return temp_dict

def convert_nested_structs(Behav_data):
    # Nested structures are loaded in strangly so use Francesca's function to extract the important ones:
    if 'RawEvents' in Behav_data['SessionData']: #dont process bad data
        Current_data = Behav_data['SessionData']['RawEvents']['Trial']
        for trial_num, trial in enumerate(Current_data,1):
            Current_data[trial_num-1] = todict(Current_data[trial_num-1])

In [ ]:
def Import_Bpod_DataFiles(InputPath):
    
    '''
    Load in all '.mat' files for a given folder and convert them to python format:
    '''
    Behav_Path = sorted(os.listdir(InputPath))
    Behav_Data = {} #set up file dict
    File_dates = []
    Sessions = 0 # for naming each data set within the main dict

    for file in Behav_Path:
        if file[-2] == 'a': #if its a .mat and not a fig
            if os.stat(InputPath+ file).st_size > 200000: #more thann 200kb 
                if file != '.DS_Store': #if file is not the weird hidden file 
                    print(file)
                    Current_file = loadmat(InputPath + file)
                    Behav_Data[Sessions] = Current_file
                    Sessions = Sessions + 1
                    File_dates = File_dates + [file[-19:-4]]
    
    return Behav_Data, Sessions, Behav_Path,File_dates